In [ ]:
import csv
import pickle
import json

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from typing import Set, Any
from ipywidgets import interact, interact_manual
from sklearn.utils import shuffle


from generation_tools.generator import QAGenerator
from generation_tools.generation import *

from generation_tools.utils.preprocessing import *
from generation_tools.utils.postprocessing import *
from generation_tools.utils.dict import *
from generation_tools.utils.load_save import *
from generation_tools.utils.split import *
from generation_tools.utils.nlp import *
from visualization_tools.qa_visualization import *

RANDOM_SEED = 626
np.random.seed(RANDOM_SEED)

## 1. Video Representation

### Load Data

In [ ]:
# Path Config
data_dir = '/root/user/' #'# plase replace following data path to your local path'
video_dir= data_dir + 'Charades/'
frame_dir = data_dir + 'STAR/frames/Charades_v1_480/'
annotation_dir =  data_dir + 'STAR/annotations/'
save_dir = data_dir+'/visualization_tmp/'
fps_filepath = annotation_dir + 'fps'    
# segment_path = annotation_dir + 'segment.json'
multi_person_vid = annotation_dir +'multi_person_video_id.json'
save_data_dir = data_dir + 'save_dir' #'# plase replace following data path to your local path' 

In [ ]:
# Load Charades/ActionGenome videos and annotations
# Load Videos
video_data = load_csv(annotation_dir)
video_data = video_data.dropna()

# Load Annotations
act_map = load_action_mapping(annotation_dir)
obj_map = load_obj_mapping(annotation_dir)
verb_map = load_verb_mapping(annotation_dir)

act_df = pd.DataFrame.from_dict(act_map, orient='index', columns = ['action_cls'] )
obj_df = pd.DataFrame.from_dict(obj_map, orient='index', columns = ['obj_cls'] )
verb_df = pd.DataFrame.from_dict(verb_map, orient='index', columns = ['verb_cls'] )

# Get Frames
fps_file = open(fps_filepath, 'rb')
fps = pickle.load(fps_file)
verb_vocab = list(set([verb_map[x] for x in verb_map.keys()]))

## segments (no need for generation)
# with open(segment_path, 'r') as f:
#    segments = json.load(f)

# Per-Obj Relationships and Object Bboxes for Video Frames 
frame_anno = load_annotations(annotation_dir)
anno_video_split = anno_data_split(frame_anno)
# anno_video_split['SYMIR']['000188']
extend_sg = json.load(open(annotation_dir + 'extend_scene_graph.json'))
sg = json.load(open(annotation_dir + 'scene_graph.json'))

### Video Filtering

In [ ]:
# Filter Config
video_filtering_cols = ['scene', 'verified', 'quality', 'relevance', 'length', 'actions_num', 'objects_num']
cols_to_drop = ['subject']
video_verb_list = 'open|close|hold|put|take|throw|grasp|turn|tidy|eat|wash|pour|drink|walk|stand|sit|lie|watch'
text_fields = 'script,descriptions'.split(',')

In [ ]:
# Filter Videos
queried_act_df = act_df[act_df['action_cls'].str.contains(video_verb_list, regex=True)] # verbs in action
filtered_videos = video_data[video_data['actions'].str.contains('|'.join(queried_act_df.index), regex=True)] 
print('The videos with useful verbs:', len(filtered_videos))
video_data = filtered_videos

video_data['objects_num'] = video_data['objects'].str.count(';')+1
video_data['actions_num'] = video_data['actions'].str.count(';')+1
video_data = filter_videos(video_data, video_filtering_cols, drop = True)

# Preprocessing action data
video_data['action_segments'] = video_data['actions'].str.split(';')

### Action Space

In [ ]:
# action space generation
def generate_action_data(video_data, verb_map, obj_map):
    action_data = video_data['actions'].str.split(';',expand=True).stack().str.split(' ',expand=True)
    action_data = action_data.rename(columns={0:'action_id', 1:'start', 2:'end'})
    action_data[['start','end']] = action_data[['start','end']].astype(float) 
    
    # sorting by starttime
    action_data.index.set_names(['idx1','idx2'],inplace=True)
    action_data = action_data.sort_values(['idx1', 'start', 'end'])
    
    # print(action_data.keys())
    action_data['action'] = action_data['action_id'].str.strip().replace(act_map, regex=True)
    action_data['verb'] = action_data['action_id'].str.strip().replace(verb_map, regex=True)
    action_data['object'] = action_data['action_id'].str.strip().replace(obj_map, regex=True)
    
    return action_data

action_data = generate_action_data(video_data, verb_map, obj_map)
print('AG original vocabulary:', action_data[['action_id','verb','object']].nunique())

In [ ]:
# action space filtering 

# filtering target verbs by verbs or actions
eliminated_verbs = 'awaken,play,undress,smile,fix,photograph'.split(',') 
conditions = ~action_data['verb'].isin(eliminated_verbs)
action_data = action_data[conditions]

eliminated_action_ids = ['a014',  
                        'a020', 'a029', 'a033',
                        'a094', 'a115', 'a131'] + ['a' + str(i) for i in range(146,157)] # c146-c156
conditions = ~action_data['action_id'].isin(eliminated_action_ids)
action_data = action_data[conditions]

action_data = action_data[~action_data['action'].str.contains( 'something', regex=True)]

# filtering condition verbs by verbs or actions
eliminated_verbs = 'undress,sneeze,awaken,snuggle,talk,photograph'.split(',') #, 'cook', 'fix', pour', 'throw', 'leave', 'start', 'begin' ]
conditions = ~action_data['verb'].isin(eliminated_verbs)
action_data = action_data[conditions]

allowed_verbs = 'open,close,hold,put,take,throw,grasp,turn,tidy,eat,wash,pour,drink,walk,stand,run,sit,lie,watch'.split(',')
conditions = action_data['verb'].isin(allowed_verbs)
action_data = action_data[conditions]

action_full_data = action_data
#---------------------------------------------
# TODO: seprate target verbs and condition verbs

condition_verb = 'open,close,hold,put,take,throw,grasp,turn,tidy,eat,wash,pour,drink,walk,stand,run,sit,lie,watch'.split(',')
conditions = action_data['verb'].isin(condition_verb)
condition_action_data = action_data[conditions]

# filtering target verbs by composition statistics
def generate_voc_by_composition(action_data):
    ## object classes statistics by verbs
    obj_classes_by_verb = action_data.groupby('verb')['object'].nunique()
    cleaned_verb_voc = obj_classes_by_verb[obj_classes_by_verb > 1] #.count()
    # cleaned_verb_voc

    ## verb classes statistics by objects
    tmp_action_data = action_data[action_data['verb'].isin(cleaned_verb_voc.keys())] # .notnull()
    verb_classes_by_obj = tmp_action_data.groupby('object')['verb'].nunique()
#     verb_classes_by_obj.pop('None')
    # print(verb_classes_by_obj)
    cleaned_obj_voc = verb_classes_by_obj[ (verb_classes_by_obj > 1) ] #.count()
    return cleaned_verb_voc, cleaned_obj_voc

cleaned_verb_voc, cleaned_obj_voc = generate_voc_by_composition(action_data)

del cleaned_obj_voc['vacuum']

action_data = action_data[action_data['verb'].isin(cleaned_verb_voc.keys())]
action_data = action_data[action_data['object'].isin(cleaned_obj_voc.keys())]

print('final vocabulary:')
print('cleaned_verb_voc',len(cleaned_verb_voc.keys()), '\t', ', '.join(cleaned_verb_voc.keys()))
print('cleaned_obj_voc',len(cleaned_obj_voc.keys()), '\t', ', '.join(cleaned_obj_voc.keys()))

keep_action_ids = action_data['action_id'].unique()
unwanted = set(act_map.keys()) - set(keep_action_ids)
# list(map(verb_map.pop, unwanted))
# list(map(obj_map.pop, unwanted))
# list(map(act_map.pop, unwanted))
print('cleaned_act_voc',len(act_map.keys()))

#---------------------------------------------
# TODO: seprate target verbs and condition verbs
target_verb =    'open,close,hold,put,take,throw,grasp,tidy,eat,wash,pour,drink,walk,sit,lie'.split(',')
conditions = action_data['verb'].isin(target_verb)
action_data = action_data[conditions]

In [ ]:
verb_list=list(set(action_full_data['verb']))
obj_list = list(set(action_full_data['object']))

### Add action columns to video_data

In [ ]:
# add ordered data by action_full_data
video_data['ordered_action_ids'] = action_full_data.groupby(level=0).agg(','.join)['action_id']
video_data['verbs'] = video_data['ordered_action_ids'].str.strip().replace(verb_map, regex=True)
video_data['objs'] = video_data['ordered_action_ids'].str.strip().replace(obj_map, regex=True)
video_data['action_words'] = video_data['ordered_action_ids'].str.strip().replace(act_map, regex=True)

In [ ]:
# add controled_action_words
video_data['controled_action_ids'] = action_data.groupby(level=0).agg(','.join)['action_id']
video_data['controled_action_words'] = video_data['controled_action_ids'].str.strip().replace(act_map, regex=True)

In [ ]:
import copy
# Clean video_data 
processed_video_data = filter_video_samples(video_data)
processed_video_data = processed_video_data.dropna()
processed_video_data_fea = copy.deepcopy(processed_video_data)

eliminated_video_ids = 'T5ECU,AAH6R,AF8I2,GL2JW,YSE1G,L3ZRP,T56KO,SBI6Z,1FX8Q,3SKWW,YN3AA,C10FA,OZIJ7,DV31C,TANB8,MUK89,BLWIW,R5L98'.split(',') 
mp_vid = json.load(open(multi_person_vid))
eliminated_video_ids = eliminated_video_ids+mp_vid
processed_video_data = processed_video_data[ ~processed_video_data['id'].isin(eliminated_video_ids) ]

In [ ]:
# Use extract_action_dict to filter action_ids in QA generation
processed_action_dict = extract_action_dict(processed_video_data, verb_map, obj_map,action_data)
processed_condition_action_dict = extract_action_dict(processed_video_data, verb_map, obj_map,condition_action_data)

In [ ]:
eliminated_video_ids = 'T5ECU,AAH6R,AF8I2,GL2JW,YSE1G,L3ZRP,T56KO,SBI6Z,1FX8Q,3SKWW,YN3AA,C10FA,BLWIW,WTJQ0,9FH1E'.split(',') 
eliminated_video_ids = eliminated_video_ids + mp_vid
processed_video_data_fea = processed_video_data_fea[ ~processed_video_data_fea['id'].isin(eliminated_video_ids) ]

In [ ]:
processed_action_dict_fea = extract_action_dict(processed_video_data_fea, verb_map, obj_map,action_data)
processed_condition_action_dict_fea = extract_action_dict(processed_video_data_fea, verb_map, obj_map,condition_action_data)


In [ ]:
list(set(action_data[(action_data['verb'] == 'eat')]['object'])-set(['book']))

## 2. QA Template and Program

In [ ]:
# CHECK: separate target and condition action
qa_templates = {}
def QAGeneration(QAGenerator, processed_action_dict, processed_condition_action_dict, avoid, qtype, template,option_generation, **kwargs):
    QAList = []
    question_idx = 0
    qa_generators = {}
    
    qa_generators["Interaction"] = [interaction_t1, interaction_t2, interaction_t3, interaction_t4]
    qa_generators["Sequence"] = [sequence_t1, sequence_t2, sequence_t3, sequence_t4, sequence_t5,sequence_t6] 
    qa_generators["Prediction"] = [prediction_t1, prediction_t2, prediction_t3, prediction_t4]
    qa_generators["Feasibility"] = [feasibility_t1, feasibility_t2, feasibility_t3, feasibility_t4, feasibility_t5,feasibility_t6]
    
    for video_id in processed_action_dict.keys():
        if not anno_video_split.__contains__(video_id):
            continue
        # all actions
        action_list = processed_action_dict[video_id][0]  
        # denoise action lsit
        if qtype!='Feasibility':
            action_list = denoise(action_list,verb_map,obj_map,**kwargs)
        if video_id not in processed_condition_action_dict.keys():
            continue
        condition_action_list = processed_condition_action_dict[video_id][0]
        template_id = '{}_T{}'.format(qtype, str(template))
        QAList, question_idx, Question, Answer = qa_generators[qtype][template-1](QAGenerator, template_id, QAList, action_list, condition_action_list, question_idx, video_id,option_generation) 
    #print(len(QAList))
    question_type = qtype +'_T' +str(template)
    if qtype not in qa_templates:
            qa_templates[qtype] = {}
    qa_templates[qtype][template-1] = {'question': Question, 'answer': Answer}
    
    if len(QAList) > 0:
        print('TID:', template_id, ', Last QID:', ('_').join([template_id, str(question_idx)]))
    else:
        print('len(QAList) == 0')
    
    return QAList, question_idx

## 3. Answer Generation

In [ ]:
kwargs = {'max_time': 10.0, 'min_time': -1, 'max_action_id': 145,
          'eliminated_action_id': [], #''.split(','),
          'specific_verb': ['hold'],
          'specific_obj': ['None'],
         } 
avoid = []

In [ ]:
# Output QA Data
save_qa_json = True
save_shuffle = False
show_shuffle = False

In [ ]:
train_answer_frequence = None
train_answer_space = None
generator = QAGenerator(processed_action_dict, anno_video_split, 
                    act_map, obj_map, verb_map, action_data, fps,
                    extend_sg, sg,
                    train_answer_space,train_answer_frequence)
option_generation=False

In [ ]:
generation_dict = {'Interaction':[1,2,3,4],
'Sequence':[1,2,3,4,5,6],
'Prediction':[1,2,3,4],
'Feasibility':[1,2,3,4,5,6]}
feasibility_filer_dict = {1:compositional_obj_feasibility,
2:compositional_verb_feasibility,3:spatial_feasibitity,4:spatial_feasibitity,
5:temporal_feasibitity,6:temporal_feasibitity}

for qtype in generation_dict:
    for temp in generation_dict[qtype]:
        if qtype=='Feasibility':
            processed_action_dict_fea_,processed_condition_action_dict_fea_ = feasibility_filer_dict[temp](processed_video_data_fea,verb_map, obj_map,action_data, condition_action_data,cleaned_obj_voc)
            QAList, _ = QAGeneration(generator, processed_action_dict_fea_,processed_condition_action_dict_fea_, avoid, qtype, temp,option_generation, **kwargs)
        else:    
            QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, avoid, qtype, temp,option_generation, **kwargs)
        
        save_qa_data(QAList, qtype, temp,save_data_dir,save_qa_json,save_shuffle)

# 4. Split and Debias

In [ ]:
qtypes = ['Interaction','Sequence','Prediction','Feasibility']
all_qa = []
for key in qtypes:
    all_qa.extend(load_all_json_in_folder(save_data_dir + key + '/'))
all_qa_meta = extract_qa_meta(all_qa)

In [ ]:
train_qa_unsampled, val_qa_unsampled, test_qa_unsampled = split_dataset(all_qa_meta,1,False,0.6,0.2,0.2)
train_ans_space = get_answer_space(train_qa_unsampled,'../annotations/generation_annotations/train_answer_space.json')
train_fre_space = get_answer_frequency(train_qa_unsampled,'../annotations/generation_annotations/train_answer_frequence.json')

In [ ]:
biased_dict = {'train':{},'val':{},'test':{}}
input_dict = {'train':train_qa_unsampled,'val':val_qa_unsampled,'test':test_qa_unsampled}
for d in input_dict:
    interaction, sequence ,prediction ,feasibility = split_by_qtype(input_dict[d])
    biased_dict[d]['Interaction']= interaction
    biased_dict[d]['Sequence']= sequence
    biased_dict[d]['Prediction']= prediction
    biased_dict[d]['Feasibility']= feasibility

In [ ]:
debised_dict = {'train':{},'val':{},'test':{}}
for d in debised_dict:
    for q in qtypes:
        debised_dict[d][q]= QA_postprocess(biased_dict[d][q],0.25,0.95,d,q)

In [ ]:
train_id, val_id, test_id = [], [], []
for q in debised_dict['train']:
    train_id.extend(debised_dict['train'][q])
for q in debised_dict['val']:
    val_id.extend(debised_dict['val'][q])
for q in debised_dict['test']:
    test_id.extend(debised_dict['test'][q])
train_id = [qa['question_id'] for qa in train_id]
val_id = [qa['question_id'] for qa in val_id]
test_id = [qa['question_id'] for qa in test_id]

In [ ]:
write_json(train_id,'../annotations/generation_annotations/train_id.json')
write_json(val_id,'../annotations/generation_annotations/val_id.json')
write_json(test_id,'../annotations/generation_annotations/test_id.json')

# 5. QA Generation

In [ ]:
option_generation=True
train_answer_frequence = json.load(open(annotation_dir+'/train_answer_frequence.json'))
train_answer_space = json.load(open(annotation_dir+'/train_answer_space.json'))
generator = QAGenerator(processed_action_dict, anno_video_split, 
                    act_map, obj_map, verb_map, action_data, fps,
                    extend_sg, sg,
                    train_answer_space,train_answer_frequence)

## Interaction T1

In [ ]:
%%time
qtype='Interaction'
template=1
selected_video_ids  = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids,show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Interaction T2 

In [ ]:
%%time
qtype='Interaction'
template=2
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids,show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Interaction T3

In [ ]:
%%time
qtype='Interaction'
template=3
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids,show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Interaction T4

In [ ]:
%%time
qtype='Interaction'
template=4
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids,show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Sequence T1

In [ ]:
%%time
qtype='Sequence'
template=1
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Sequence T2 

In [ ]:
%%time 
qtype='Sequence'
template=2
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Sequence T3 

In [ ]:
%%time 
qtype='Sequence'
template=3
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Sequence T4 

In [ ]:
%%time
qtype='Sequence'
template=4
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Sequence T5 

In [ ]:
%%time 
qtype='Sequence'
template=5
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Sequence T6

In [ ]:
%%time
qtype='Sequence'
template=6
selected_video_ids = []

# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Prediction T1

In [ ]:
%%time
qtype='Prediction'
template=1
selected_video_ids = []

# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[0:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Prediction T2

In [ ]:
%%time
qtype='Prediction'
template=2
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Prediction T3

In [ ]:
%%time
qtype='Prediction'
template=3
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Prediction T4

In [ ]:
%%time
qtype='Prediction'
template=4
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict,processed_condition_action_dict, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Feasibility T1

In [ ]:
%%time
processed_action_dict_fea_,processed_condition_action_dict_fea_ = compositional_obj_feasibility(processed_video_data_fea,verb_map, obj_map,action_data, condition_action_data,cleaned_obj_voc)
qtype='Feasibility'
template=1

selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict_fea_,processed_condition_action_dict_fea_, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Feasibility T2

In [ ]:
%%time
processed_action_dict_fea_,processed_condition_action_dict_fea_ = compositional_verb_feasibility(processed_video_data_fea,verb_map, obj_map,action_data, condition_action_data,cleaned_obj_voc)
qtype='Feasibility'
template=2
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict_fea_,processed_condition_action_dict_fea_, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Feasibility T3

In [ ]:
%%time
processed_action_dict_fea_,processed_condition_action_dict_fea_ = spatial_feasibitity(processed_video_data_fea,verb_map, obj_map,action_data, condition_action_data,cleaned_obj_voc)
qtype='Feasibility'
template=3
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict_fea_,processed_condition_action_dict_fea_, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Feasibility T4

In [ ]:
%%time
qtype='Feasibility'
template=4
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict_fea_,processed_condition_action_dict_fea_, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))

# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Feasibility T5

In [ ]:
%%time
processed_action_dict_fea_,processed_condition_action_dict_fea_ = temporal_feasibitity(processed_video_data_fea,verb_map, obj_map,action_data, condition_action_data,cleaned_obj_voc)
qtype='Feasibility'
template=5
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict_fea_,processed_condition_action_dict_fea_, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)

## Feasibility T6

In [ ]:
%%time
qtype='Feasibility'
template=6
selected_video_ids = []
# generate
QAList, _ = QAGeneration(generator, processed_action_dict_fea_,processed_condition_action_dict_fea_, 
                         avoid,qtype, template,option_generation, **kwargs)
print('generated', len(QAList))
# post processing for demo
QAList_demo = demo_postprocess(QAList, selected_video_ids, show_shuffle)
# visulization for quanlity checking
@interact
def show_examples(question_idx = range(0, len(QAList_demo[:100]))):   
    temp_qa_visulization([QAList_demo[:100][question_idx]], video_dir ,save_dir) 
    
# save output
save_qa_data(QAList, qtype, template,save_data_dir,save_qa_json,save_shuffle)